In [1]:
import torch
import torchvision
import numpy as np
import module
import vgg

In [2]:
# model = torchvision.models.vgg16(pretrained=True)
model = vgg.vgg16(pretrained=True)
model = torch.nn.DataParallel(model).cuda()

In [3]:
quantization_weight = [None for i in range(16)]
pruning_weight = [8 for i in range(16)]

In [8]:
def weight_processing(model, quantization_factor, pruning_factor):
    state_dict = model.state_dict()
        
    counter = 0
    total = 0
    sum = 0
    for module_name, layer in model.named_modules():
        if type(layer) == torch.nn.modules.conv.Conv2d \
        or type(layer) == torch.nn.modules.linear.Linear or type(layer) == torch.nn.modules.batchnorm.BatchNorm2d or type(layer) == module.QuantizeConv2d or type(layer) == module.QuantizeLinear: 
            for layer_name, parameter in layer.named_parameters():
                if quantization_factor[counter] != None:
                    state_dict[module_name + '.' + layer_name] = module.quantize(parameter, quantization_factor[counter])
                if pruning_factor != None:
                    state_dict[module_name + '.' + layer_name] = state_dict[module_name + '.' + layer_name] * module.pruning(parameter, pruning_factor[counter])
                    sum += len(torch.nonzero(state_dict[module_name + '.' + layer_name]))
                    total += len(state_dict[module_name + '.' + layer_name].view(-1))
                    print('['+str(counter)+'] layer pruning rate : ' + str((1-(len(torch.nonzero(state_dict[module_name + '.' + layer_name])) / 
                                                                               len(state_dict[module_name + '.' + layer_name].view(-1))))*100))
            counter += 1
     
    print('total layer pruning rate : ' + str((1-sum/total)*100))
            
    model.load_state_dict(state_dict)


In [9]:
# print(model.state_dict()['module.features.0.weight'][0])
weight_processing(model, quantization_weight, pruning_weight)
# print(model.state_dict()['module.features.0.weight'][0])

[0] layer pruning rate : 1.5625
[0] layer pruning rate : 0.0
[1] layer pruning rate : 8.034939236111116
[1] layer pruning rate : 1.5625
[2] layer pruning rate : 8.245171440972221
[2] layer pruning rate : 1.5625
[3] layer pruning rate : 9.608968098958337
[3] layer pruning rate : 2.34375
[4] layer pruning rate : 11.346435546875
[4] layer pruning rate : 3.125
[5] layer pruning rate : 13.766140407986116
[5] layer pruning rate : 3.515625
[6] layer pruning rate : 13.465203179253471
[6] layer pruning rate : 3.515625
[7] layer pruning rate : 15.970272488064241
[7] layer pruning rate : 5.078125
[8] layer pruning rate : 19.915262858072914
[8] layer pruning rate : 3.90625
[9] layer pruning rate : 19.91098192003038
[9] layer pruning rate : 4.4921875
[10] layer pruning rate : 18.678453233506943
[10] layer pruning rate : 2.734375
[11] layer pruning rate : 18.590545654296875
[11] layer pruning rate : 3.125
[12] layer pruning rate : 20.757039388020836
[12] layer pruning rate : 2.34375
[13] layer pruni

In [16]:
torch.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

In [30]:
len(temp.view(-1))

27